# Summary
In this episode, we used database JOIN operations to select photometry data for the stars we’ve identified as candidates to be in GD-1

The following figure from the Price-Whelan and Bonaca paper is a color-magnitude diagram for the stars selected based on proper motion:

<img src="/Users/darykevin/Documents/Python Scripts/Computational Cosmology/Project GD 1/isochrone.png" alt="Alternative text" width="1100" height="400" />

In red is a stellar [isochrone](https://en.wikipedia.org/wiki/Stellar_isochrone), showing where we expect the stars in GD-1 to fall based on the metallicity and age of their original globular cluster.

By selecting stars in the shaded area, we can further distinguish the main sequence of GD-1 from younger background stars.

In [21]:
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
import astropy.units as u
from gala.coordinates import GD1Koposov10, GD1, reflex_correct
import pandas as pd

In [2]:
filename = 'gd1_data.hdf'
point_series = pd.read_hdf(filename, 'point_series')
sky_point_list = point_series['sky_point_list']
pmra_min = point_series['pmra_min']
pmra_max = point_series['pmra_max']
pmdec_min = point_series['pmdec_min']
pmdec_max = point_series['pmdec_max']
point_series

sky_point_list    135.306, 8.39862, 126.51, 13.4449, 163.017, 54...
pmra_min                                                       -6.7
pmra_max                                                         -3
pmdec_min                                                    -14.31
pmdec_max                                                     -11.2
dtype: object

# Getting photometry data

The Gaia dataset contains some photometry data, including the variable `bp_rp`, which contains BP-RP color (the difference in mean flux between the BP and RP bands). We use this variable to select stars with `bp_rp` between -0.75 and 2, which excludes many class M dwarf stars.

But we can do better than that. Assuming GD-1 is a globular cluster, all of the stars formed at the same time from the same material, so the stars’ photometric properties should be consistent with a single isochrone in a color magnitude diagram. We can use photometric color and apparent magnitude to select stars with the age and metal richness we expect in GD-1. However, the broad Gaia photometric bands (G, BP, RP) are not optimized for this task, instead we will use the more narrow photometric bands available from the Pan-STARRS survey to obtain the `g-i` color and apparent `g`-band magnitude.

Conveniently, the Gaia server provides data from Pan-STARRS as a table in the same database we have been using, so we can access it by making ADQL queries.

Documentation for Cross-Matching: [LINK](https://gea.esac.esa.int/archive/documentation/GDR2/Catalogue_consolidation/chap_cu9val_cu9val/ssec_cu9xma/sssec_cu9xma_extcat.html)

# The best neighbor table

In this example:

* Stars in the Gaia dataset are identified by `source_id`.

* Stars in the Pan-STARRS dataset are identified by `obj_id`.

For each candidate star we have selected so far, we have the source_id; the goal is to find the `obj_id` for the same star in the Pan-STARRS catalog.

To do that we will:

Use the `JOIN` operator to look up each Pan-STARRS `obj_id` for the stars we are interested in in the `panstarrs1_best_neighbour` table using the source_ids that we have already identified.

Use the `JOIN` operator again to look up the Pan-STARRS photometry for these stars in the `panstarrs1_original_valid` table using the `obj_ids` we just identified.

Before we get to the JOIN operation, we will explore these tables.

In [3]:
ps_best_neighbour_meta = Gaia.load_table('gaiadr2.panstarrs1_best_neighbour')

Retrieving table 'gaiadr2.panstarrs1_best_neighbour'


In [4]:
print(ps_best_neighbour_meta)

TAP Table name: gaiadr2.gaiadr2.panstarrs1_best_neighbour
Description: Pan-STARRS1 BestNeighbour table lists each matched Gaia object with its
best neighbour in the external catalogue.
There are 1 327 157 objects in the filtered version of Pan-STARRS1 used
to compute this cross-match that have too early epochMean.
Num. columns: 7


In [5]:
# And here are the columns.
for column in ps_best_neighbour_meta.columns:
    print(column.name)

source_id
original_ext_source_id
angular_distance
number_of_neighbours
number_of_mates
best_neighbour_multiplicity
gaia_astrometric_params


Documentation for the above headings: [LINK](https://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_crossmatches/ssec_dm_panstarrs1_best_neighbour.html)

The ones we will use are:

`source_id`, which we will match up with `source_id` in the Gaia table.

`best_neighbour_multiplicity`, which indicates how many sources in Pan-STARRS are matched with the same probability to this source in Gaia.

`number_of_mates`, which indicates the number of other sources in Gaia that are matched with the same source in Pan-STARRS.

``original_ext_source_id``, which we will match up with ``obj_id`` in the Pan-STARRS table.

Ideally, `best_neighbour_multiplicity` should be 1 and number_of_mates should be 0; in that case, there is a one-to-one match between the source in Gaia and the corresponding source in Pan-STARRS.

Here is a query that selects these columns and returns the first 5 rows

# The Pan-STARRS table

Now that we know the Pan-STARRS `obj_id`, we are ready to match this to the photometry in the `panstarrs1_original_valid` table. Here is the metadata for the table that contains the Pan-STARRS data.

In [6]:
ps_valid_meta = Gaia.load_table('gaiadr2.panstarrs1_original_valid')

Retrieving table 'gaiadr2.panstarrs1_original_valid'


In [7]:
print(ps_valid_meta)

TAP Table name: gaiadr2.gaiadr2.panstarrs1_original_valid
Description: The Panoramic Survey Telescope and Rapid Response System (Pan-STARRS) is
a system for wide-field astronomical imaging developed and operated by
the Institute for Astronomy at the University of Hawaii. Pan-STARRS1
(PS1) is the first part of Pan-STARRS to be completed and is the basis
for Data Release 1 (DR1). The PS1 survey used a 1.8 meter telescope and
its 1.4 Gigapixel camera to image the sky in five broadband filters (g,
r, i, z, y).

The current table contains a filtered subsample of the 10 723 304 629
entries listed in the original ObjectThin table.
We used only ObjectThin and MeanObject tables to extract
panstarrs1OriginalValid table, this means that objects detected only in
stack images are not included here. The main reason for us to avoid the
use of objects detected in stack images is that their astrometry is not
as good as the mean objects astrometry: “The stack positions (raStack,
decStack) have considera

In [8]:
for column in ps_valid_meta.columns:
    print(column.name)

obj_name
obj_id
ra
dec
ra_error
dec_error
epoch_mean
g_mean_psf_mag
g_mean_psf_mag_error
g_flags
r_mean_psf_mag
r_mean_psf_mag_error
r_flags
i_mean_psf_mag
i_mean_psf_mag_error
i_flags
z_mean_psf_mag
z_mean_psf_mag_error
z_flags
y_mean_psf_mag
y_mean_psf_mag_error
y_flags
n_detections
zone_id
obj_info_flag
quality_flag


Documentation for the above headings: [LINK](https://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_external_catalogues/ssec_dm_panstarrs1_original_valid.html)

The ones we will use are:

* `obj_id`, which we will match up with original_ext_source_id in the best neighbor table.

* `g_mean_psf_mag`, which contains mean magnitude from the g filter.

* `i_mean_psf_mag`, which contains mean magnitude from the i filter.

# Joining Tables

The following figure shows how these tables are related.

* The orange circles and arrows represent the first `JOIN` operation, which takes each `source_id` in the Gaia table and finds the same value of `source_id` in the best neighbor table.

* The blue circles and arrows represent the second `JOIN` operation, which takes each `original_ext_source_id` in the Gaia table and finds the same value of `obj_id` in the best neighbor table.

<img src="/Users/darykevin/Documents/Python Scripts/Computational Cosmology/Project GD 1/join.png" alt="Alternative text" width="1000" height="600" />

There is no guarantee that the corresponding rows of these tables are in the same order, so the `JOIN` operation involves some searching. However, ADQL/SQL databases are implemented in a way that makes this kind of search efficient. If you are curious, [you can read more about it](https://chartio.com/learn/databases/how-does-indexing-work/).

# Selecting by coordinates and proper motion

In [11]:
columns = 'source_id, ra, dec, pmra, pmdec'

candidate_join_query_base = """
SELECT 
{columns}
FROM gaiadr2.gaia_source as gaia
JOIN gaiadr2.panstarrs1_best_neighbour as best
  ON gaia.source_id = best.source_id
JOIN gaiadr2.panstarrs1_original_valid as ps
  ON best.original_ext_source_id = ps.obj_id
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2 
  AND 1 = CONTAINS(POINT(gaia.ra, gaia.dec), 
                   POLYGON({sky_point_list}))
  AND gaia.pmra BETWEEN {pmra_min} AND  {pmra_max}
  AND gaia.pmdec BETWEEN {pmdec_min} AND {pmdec_max}
"""
column_list = ['gaia.source_id',
               'gaia.ra',
               'gaia.dec',
               'gaia.pmra',
               'gaia.pmdec',
               'best.best_neighbour_multiplicity',
               'best.number_of_mates',
               'ps.g_mean_psf_mag',
               'ps.i_mean_psf_mag']

columns = ', '.join(column_list)

candidate_join_query = candidate_join_query_base.format(columns=columns,
                            sky_point_list= sky_point_list,
                            pmra_min=pmra_min,
                            pmra_max=pmra_max,
                            pmdec_min=pmdec_min,
                            pmdec_max=pmdec_max)
print(candidate_join_query)


candidate_join_job = Gaia.launch_job_async(candidate_join_query)
candidate_table = candidate_join_job.get_results()
candidate_table


SELECT 
gaia.source_id, gaia.ra, gaia.dec, gaia.pmra, gaia.pmdec, best.best_neighbour_multiplicity, best.number_of_mates, ps.g_mean_psf_mag, ps.i_mean_psf_mag
FROM gaiadr2.gaia_source as gaia
JOIN gaiadr2.panstarrs1_best_neighbour as best
  ON gaia.source_id = best.source_id
JOIN gaiadr2.panstarrs1_original_valid as ps
  ON best.original_ext_source_id = ps.obj_id
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2 
  AND 1 = CONTAINS(POINT(gaia.ra, gaia.dec), 
                   POLYGON(135.306, 8.39862, 126.51, 13.4449, 163.017, 54.2424, 172.933, 46.4726, 135.306, 8.39862))
  AND gaia.pmra BETWEEN -6.7 AND  -3
  AND gaia.pmdec BETWEEN -14.31 AND -11.2

INFO: Query finished. [astroquery.utils.tap.core]


source_id,ra,dec,pmra,pmdec,best_neighbour_multiplicity,number_of_mates,g_mean_psf_mag,i_mean_psf_mag
,deg,deg,mas / yr,mas / yr,,,,mag
int64,float64,float64,float64,float64,int16,int16,float64,float64
635860218726658176,138.5187065217173,19.09233926905897,-5.941679495793577,-11.346409129876392,1,0,17.8978004455566,17.5174007415771
635674126383965568,138.8428741026386,19.031798198627634,-3.8970011609340207,-12.702779525389634,1,0,19.2873001098633,17.6781005859375
635614168640132864,139.59219748145836,18.807955539071433,-3.309602916796381,-13.708904908478631,1,0,16.1515998840332,14.6662998199463
635694467349085056,139.06182692172476,19.06744237567232,-3.0674889862265795,-11.4175270465402,1,0,18.6709003448486,17.9841995239258
635598607974369792,139.20920023089508,18.624132868942702,-6.124445176881091,-12.833824027100611,1,0,16.5223999023438,16.1375007629395
635850945892748672,139.86542888472115,20.011312663154804,-3.786655365804428,-14.28415600718206,1,0,16.5174999237061,16.0450000762939
635600532119713664,139.22869949616816,18.685939084485494,-3.9742788217925122,-12.342426623384245,1,0,20.4505996704102,19.5177001953125
635620701286087424,139.6862822364899,18.881113520820737,-3.3558418379356474,-12.925224236202752,1,0,20.4113006591797,19.5032997131348


# Checking the match

To get more information about the matching process, we can inspect `best_neighbour_multiplicity`, which indicates for each star in Gaia how many stars in Pan-STARRS are equally likely matches.

In [12]:
candidate_table['best_neighbour_multiplicity']

1
1
1
1
1
1
1
1
1
1
1


Most of the values are `1`, which is good; that means that for each candidate star we have identified exactly one source in Pan-STARRS that is likely to be the same star.

To check whether there are any values other than 1, we can convert this column to a Pandas `Series` and use `describe`

In [13]:
multiplicity = pd.Series(candidate_table['best_neighbour_multiplicity'])
multiplicity.describe()

count    4300.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
dtype: float64

Similarly, `number_of_mates` indicates the number of other stars in Gaia that match with the same star in Pan-STARRS.

In [14]:
mates = pd.Series(candidate_table['number_of_mates'])
mates.describe()

count    4300.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
dtype: float64

All values in this column are `0`, which means that for each match we found in Pan-STARRS, there are no other stars in Gaia that also match.

# Saving the DataFrame

In [15]:
def make_dataframe(table):
    """Transform coordinates from ICRS to GD-1 frame.
    
    table: Astropy Table
    
    returns: Pandas DataFrame
    """
    #Create a SkyCoord object with the coordinates and proper motions
    # in the input table
    skycoord = SkyCoord(
               ra=table['ra'], 
               dec=table['dec'],
               pm_ra_cosdec=table['pmra'],
               pm_dec=table['pmdec'], 
               distance=8*u.kpc, 
               radial_velocity=0*u.km/u.s)

    # Define the GD-1 reference frame
    gd1_frame = GD1Koposov10()

    # Transform input coordinates to the GD-1 reference frame
    transformed = skycoord.transform_to(gd1_frame)

    # Correct GD-1 coordinates for solar system motion around galactic center
    skycoord_gd1 = reflex_correct(transformed)

    #Add GD-1 reference frame columns for coordinates and proper motions
    table['phi1'] = skycoord_gd1.phi1
    table['phi2'] = skycoord_gd1.phi2
    table['pm_phi1'] = skycoord_gd1.pm_phi1_cosphi2
    table['pm_phi2'] = skycoord_gd1.pm_phi2

    # Create DataFrame
    df = table.to_pandas()

    return df

In [22]:
candidate_df = make_dataframe(candidate_table)

In [23]:
filename = 'gd1_data.hdf'

candidate_df.to_hdf(filename, 'candidate_df')

In [24]:
from os.path import getsize

MB = 1024 * 1024
getsize(filename) / MB

15.422508239746094